Using logistic regression (& other methods) to try to predict:
* rank of the image; given sentence, compound & caption
* idiomatic / literal; given sentence, (compound) & ChatGPT description

It seems like these methods do not produce satisfactory results.

In [ ]:
import csv
import sys
import pandas as pd
import ast

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Code von Wiebke
fileName = "subtask_a_train.tsv"
fileDirectory = "AdMIRe Subtask A Train/train"
data = pd.read_csv(fileDirectory + "/" + fileName, sep='\t')
data['expected_order'] = data['expected_order'].apply(ast.literal_eval)

# descriptions given by ChatGPT for idiomatic / literal use of compound
# read data von Julio -> csv file
filename_desc = "gpt-desc.csv"
data_desc = pd.read_csv(filename_desc)

Predicting rank

In [114]:
# ceate training data
# X = matrix (# samples, # features) -> features = combination of embeddings for compound, sentence, caption
# y = array (# samples)

import numpy as np

X = []
y = []

for i in range(len(data)):

    current = data.iloc[i]

    # add input & place in ranking for image 1
    sentences = [current["compound"],
                current["sentence"], 
                current["image1_caption"]]
    embeddings = model.encode(sentences)
    X.append(np.concatenate(((embeddings[0] + embeddings[1]) / 2, embeddings[2])))
    #X.append(np.concatenate((embeddings[0], embeddings[1], embeddings[2])))
    #X.append(embeddings) -> doesn't work

    if current["image1_name"] == current["expected_order"][0]:
        y.append(1)
    elif current["image1_name"] == current["expected_order"][1]:
        y.append(2)
    elif current["image1_name"] == current["expected_order"][2]:
        y.append(3)
    elif current["image1_name"] == current["expected_order"][3]:
        y.append(4)
    else:
        y.append(5)

    # for image 2
    sentences = [current["compound"],
                current["sentence"], 
                current["image2_caption"]]
    embeddings = model.encode(sentences)
    X.append(np.concatenate(((embeddings[0] + embeddings[1]) / 2, embeddings[2])))
    #X.append(np.concatenate((embeddings[0], embeddings[1], embeddings[2])))

    if current["image2_name"] == current["expected_order"][0]:
        y.append(1)
    elif current["image2_name"] == current["expected_order"][1]:
        y.append(2)
    elif current["image2_name"] == current["expected_order"][2]:
        y.append(3)
    elif current["image2_name"] == current["expected_order"][3]:
        y.append(4)
    else:
        y.append(5)

    # for image 3
    sentences = [current["compound"],
                current["sentence"], 
                current["image3_caption"]]
    embeddings = model.encode(sentences)
    X.append(np.concatenate(((embeddings[0] + embeddings[1]) / 2, embeddings[2])))
    #X.append(np.concatenate((embeddings[0], embeddings[1], embeddings[2])))

    if current["image3_name"] == current["expected_order"][0]:
        y.append(1)
    elif current["image3_name"] == current["expected_order"][1]:
        y.append(2)
    elif current["image3_name"] == current["expected_order"][2]:
        y.append(3)
    elif current["image3_name"] == current["expected_order"][3]:
        y.append(4)
    else:
        y.append(5)

    # for image 4
    sentences = [current["compound"],
                current["sentence"], 
                current["image4_caption"]]
    embeddings = model.encode(sentences)
    X.append(np.concatenate(((embeddings[0] + embeddings[1]) / 2, embeddings[2])))
    #X.append(np.concatenate((embeddings[0], embeddings[1], embeddings[2])))

    if current["image4_name"] == current["expected_order"][0]:
        y.append(1)
    elif current["image4_name"] == current["expected_order"][1]:
        y.append(2)
    elif current["image4_name"] == current["expected_order"][2]:
        y.append(3)
    elif current["image4_name"] == current["expected_order"][3]:
        y.append(4)
    else:
        y.append(5)
    
    # for image 5
    sentences = [current["compound"],
                current["sentence"], 
                current["image5_caption"]]
    embeddings = model.encode(sentences)
    X.append(np.concatenate(((embeddings[0] + embeddings[1]) / 2, embeddings[2])))
    #X.append(np.concatenate((embeddings[0], embeddings[1], embeddings[2])))

    if current["image5_name"] == current["expected_order"][0]:
        y.append(1)
    elif current["image5_name"] == current["expected_order"][1]:
        y.append(2)
    elif current["image5_name"] == current["expected_order"][2]:
        y.append(3)
    elif current["image5_name"] == current["expected_order"][3]:
        y.append(4)
    else:
        y.append(5)


In [115]:
# create version of y for determining just whether or not top1

y_top1 = []
for i in y:
    if i == 1:
        y_top1.append(1)
    else:
        y_top1.append(0)

In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
#X_train, X_test, y_train, y_test = train_test_split(X, y_top1)

In [117]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


In [118]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

log_reg = LogisticRegression().fit(X_train, y_train)

# to see how well it fits X_train & y_train:
# print(log_reg.score(X_train, y_train))

y_pred = log_reg.predict(X_test)
#print(y_pred)
#print(accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred, zero_division=0.0))

              precision    recall  f1-score   support

           1       0.16      0.24      0.19        17
           2       0.08      0.30      0.13        10
           3       0.00      0.00      0.00        20
           4       0.10      0.05      0.07        20
           5       0.33      0.19      0.24        21

    accuracy                           0.14        88
   macro avg       0.13      0.16      0.13        88
weighted avg       0.14      0.14      0.12        88



In [119]:
from sklearn import svm

svm_reg = svm.SVR()
svm_reg.fit(X_train, y_train)
svm_reg.score(X_test, y_test)

0.03675762247574155

In [120]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier().fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0.0))

              precision    recall  f1-score   support

           1       0.24      0.24      0.24        17
           2       0.07      0.20      0.10        10
           3       0.25      0.20      0.22        20
           4       0.33      0.30      0.32        20
           5       0.50      0.19      0.28        21

    accuracy                           0.23        88
   macro avg       0.28      0.23      0.23        88
weighted avg       0.31      0.23      0.25        88



In [121]:
# random baseline (performs better than anything above)

from random import randrange

y_pred = []
for i in range(88):
    y_pred.append(randrange(1,6))

print(classification_report(y_test, y_pred, zero_division=0.0))

              precision    recall  f1-score   support

           1       0.19      0.24      0.21        17
           2       0.19      0.30      0.23        10
           3       0.25      0.15      0.19        20
           4       0.32      0.30      0.31        20
           5       0.25      0.24      0.24        21

    accuracy                           0.24        88
   macro avg       0.24      0.24      0.24        88
weighted avg       0.25      0.24      0.24        88



Predicting idiomatic / literal

In [150]:
# ceate training data
# X = matrix (# samples, # features) -> features: combination of embeddings for compound, (sentence), literal / idiomatic description
# y = array (# samples)

X = []
y = []

for i in range(len(data)):

    current = data.iloc[i]

    # determine whether or not literal
    #description = data_desc[(data_desc["compound"]==current["compound"]) 
    #                        & (data_desc["sentence_type"]=="literal")]["Meaning"].item()
    
    # determine whether or not idiomatic
    description = data_desc[(data_desc["compound"]==current["compound"]) 
                            & (data_desc["sentence_type"]=="idiomatic")]["Meaning"].item()
    
    sentences = [current["compound"],
                current["sentence"], 
                description]
    
    embeddings = model.encode(sentences)

    X.append(np.concatenate(((embeddings[0] + embeddings[1]) / 2, embeddings[2])))
    #X.append(np.concatenate((embeddings[1], embeddings[2])))

    if current["sentence_type"] == "literal":
        y.append(1)
    else:
        y.append(0)

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [159]:
log_reg = LogisticRegression().fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0.0))

              precision    recall  f1-score   support

           0       0.62      1.00      0.77        10
           1       1.00      0.25      0.40         8

    accuracy                           0.67        18
   macro avg       0.81      0.62      0.58        18
weighted avg       0.79      0.67      0.61        18



In [153]:
clf = RandomForestClassifier().fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0.0))

              precision    recall  f1-score   support

           0       0.56      0.90      0.69        10
           1       0.50      0.12      0.20         8

    accuracy                           0.56        18
   macro avg       0.53      0.51      0.45        18
weighted avg       0.53      0.56      0.47        18



In [158]:
svm_reg = svm.SVR().fit(X_train, y_train)
svm_reg.score(X_test, y_test)

0.07521790391705974